In [ ]:
# リッチな見た目のグラフライブラリ
import seaborn as sns
import gc

In [ ]:
# gcを使えるようにする.
gc.enable()
# del hoge
# gc.collect() で使える

# グラフの見た目をいい感じにする
sns.set()

# 住宅価格の予想でこんにちは

## データのダウンロード

In [ ]:
from sklearn.datasets import load_boston

なにこれって関数の中身とか知りたいときは...

In [ ]:
load_boston?

In [ ]:
boston = load_boston()

## 特徴量の確認

In [ ]:
boston.data

## 答えの確認

In [ ]:
boston.target

In [ ]:
boston.feature_names

詳しくはこちら(https://scikit-learn.org/stable/datasets/index.html#boston-dataset)  
日本語ではこちら(https://pythondatascience.plavox.info/scikit-learn/scikit-learn%E3%81%AB%E4%BB%98%E5%B1%9E%E3%81%97%E3%81%A6%E3%81%84%E3%82%8B%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88)

# pandasを使って見やすくデータ分析をする

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(boston.data, columns=boston.feature_names)

## いろいろできます

### 相関係数

In [ ]:
df.corr()

### AGEが50以上のデータ抽出

In [ ]:
df.query('AGE > 50')

### 欠損値の確認

In [ ]:
# このデータに欠損値はない
df.isna().sum() / df.shape[0]

# 学習するための準備

予測するためのデータについて考えます.  
- boston.dataには学習するための情報(boston.data)と答え(boston.target)がある.
- 実際に必要なのは未知データに対しての予測値.
- データを学習する部分と,モデルの性能を評価する部分に分ける必要がある.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# もとのデータの形
boston.data.shape

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(boston.data, boston.target, test_size=0.3, random_state=42)

In [ ]:
# 学習用のデータの形
train_X.shape

In [ ]:
# test用のデータの形
test_X.shape

# 線形回帰

 $y = w_0 + w_1x_1 + w_2x_2 + \cdots + w_nx_n$

In [ ]:
#x1, x2などは各列(特徴量)の値です.
df.head(1)

学習するのは$w_i$の部分です.

In [ ]:
from sklearn.linear_model import LinearRegression

scikit-learnのAPIの流れは,
1. モデルのインスタンスの作成
1. fitメソッドで学習
1. predictメソッドで予測

です.

In [ ]:
# モデルのインスタンスを作成
model = LinearRegression()

In [ ]:
# 学習
model.fit(train_X, train_y)

In [ ]:
# 予測
predict = model.predict(test_X)

## モデルの精度検証

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(test_y, predict) ** 0.5

4638ドルのズレ☆

## 重みの可視化

重み$w_i$がどの様になっているか可視化する

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_weight(model):
    fig, ax = plt.subplots(figsize=(16, 8))
    x = list(range(model.coef_.size))
    y = model.coef_
    ax.bar(x, y)
    ax.set_xticklabels(boston.feature_names) 
    

figやらaxやらについて詳しく知りたい人向け(https://qiita.com/skotaro/items/08dc0b8c5704c94eafb9)

In [ ]:
plot_weight(model)

# データのスケールを合わせる 

In [ ]:
df.median().plot.bar(figsize=(16, 8))

## データの標準化

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
transformer = StandardScaler()

In [ ]:
train_X_norm = transformer.fit_transform(train_X)

In [ ]:
test_X_norm = transformer.transform(test_X)

In [ ]:
model = LinearRegression()
model.fit(train_X_norm, train_y)
predict = model.predict(test_X_norm)

In [ ]:
mean_squared_error(test_y, predict) ** 0.5

In [ ]:
plot_weight(model)

In [ ]:
del model, transformer
gc.collect()

## データの正規化

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
transformer = Normalizer()

In [ ]:
train_X_norm = transformer.fit_transform(train_X)

In [ ]:
test_X_norm = transformer.transform(test_X)

In [ ]:
model = LinearRegression()
model.fit(train_X_norm, train_y)
predict = model.predict(test_X_norm)

In [ ]:
mean_squared_error(test_y, predict) ** 0.5

In [ ]:
plot_weight(model)

In [ ]:
del model, transformer
gc.collect()

## 同じことを何回もするの大変なのでこれ以降はPipeLine化する

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
pl = make_pipeline(Normalizer(), LinearRegression())

In [ ]:
pl.fit(train_X, train_y)
predict_linear = pl.predict(test_X)
mean_squared_error(test_y, predict_linear) ** 0.5

# 正則化項をいれる

In [ ]:
from sklearn.linear_model import Ridge, Lasso

## Ridge

In [ ]:
pl = make_pipeline(Normalizer(), Ridge(alpha=1e-6))
pl.fit(train_X, train_y)
predict_ridge = pl.predict(test_X)
mean_squared_error(test_y, predict_ridge) ** 0.5

In [ ]:
plot_weight(pl.named_steps['ridge'])

## Lasso

In [ ]:
pl = make_pipeline(Normalizer(), Lasso(alpha=1e-6))
pl.fit(train_X, train_y)
predict_lasso = pl.predict(test_X)
mean_squared_error(test_y, predict_lasso) ** 0.5

In [ ]:
plot_weight(pl.named_steps['lasso'])

# まとめ

- まずはどんなデータかを理解する
- 機械にデータを理解させるための前処理をする
- 良さげな特徴量をつくる
- 過学習と学習不足に気を配る